In [1]:
import csv
import json
import pandas as pd
import re
import logging
import os
import random
import sys
from heapq import nlargest
import openai
import torch
from tenacity import retry, stop_after_attempt, wait_exponential
import pandas as pd


with open('data_set_v3.0/author_json.json', 'r',encoding='utf-8') as f:
    authors_sampled = json.load(f)
print(len(authors_sampled))

with open('data_set_v3.0/paper_json.json', 'r',encoding='utf-8') as f:
    papers_sampled = json.load(f)
print(len(papers_sampled))


281
107


In [5]:
input_history = pd.read_csv('naive_method_v3.0/paper_to_author_cot_5_examples.tsv', sep='\t', header=None, names=["paper_index","author_index","example_index","result","abstract","title"])
user_keywords_topics = pd.read_csv('naive_method_v3.0/author_profiling_20_examples_.tsv', sep='\t', header=None, names=["author_index","keywords","topics"])

id2authorTopics = {}
id2authorInfo = {}
id2ICLexamples = {}
for i in range(len(user_keywords_topics["author_index"])):
    id2authorTopics[user_keywords_topics["author_index"][i]] = {"keywords":user_keywords_topics["keywords"][i],"topics":user_keywords_topics["topics"][i]}

for i in range(len(input_history["paper_index"])):
    paperId = input_history["paper_index"][i]

    authorId = input_history["author_index"][i]
    if paperId not in id2ICLexamples:
        id2ICLexamples[paperId] = {}
    if authorId not in id2ICLexamples[paperId]:
        id2ICLexamples[paperId][authorId]=[]
    id2ICLexamples[paperId][authorId].append({"abstract":input_history["abstract"][i],"title":input_history["title"][i],"result":input_history["result"][i]})

    if authorId not in id2authorInfo:
        id2authorInfo[authorId]=[]
    id2authorInfo[authorId].append({"abstract":input_history["abstract"][i],"title":input_history["title"][i],"result":input_history["result"][i]})



# Swap author ID
import random
authorList = [i for i in id2authorTopics]
for paperId in id2ICLexamples:
    # # Swap First Author 
    # oldAuhtorId = next(iter(id2ICLexamples[paperId]))
    # Swap random author 
    oldAuhtorId =  random.choice([i for i in id2ICLexamples[paperId]])
    newAuhtorId = oldAuhtorId
    while newAuhtorId in id2ICLexamples[paperId]:
        newAuhtorId = random.choice(authorList)
    tempAuthorDict = {}
    for i,authorId in enumerate(id2ICLexamples[paperId]):
        if oldAuhtorId ==authorId:
            tempAuthorDict[newAuhtorId] = id2authorInfo[newAuhtorId]
        else:
            tempAuthorDict[authorId] = id2authorInfo[authorId]
    id2ICLexamples[paperId] = tempAuthorDict
print([i for i in id2ICLexamples["130862d54894966552cb85d3ee6f739f885d4989"]])
print(id2ICLexamples["130862d54894966552cb85d3ee6f739f885d4989"])

len(id2authorTopics)

20

In [ ]:

error_list=[]
i = 0
openai.api_key = ""
openai.api_base = "https://gersteinbiocodeeval-eastus2.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
key_bundles = [
    ('', "https://gersteinbiocodeeval-eastus2.openai.azure.com/"),
    ('', "https://gersteinbiocodeeval-eastus2.openai.azure.com/"),
    ('', "https://gersteinbiocodeeval-eastus2.openai.azure.com/")
]


    # prompt_construction v1.6
def construct_prompt_cot_agg(author_dict,authorId,paperId):
    prompt_str = ""
    keywords = ""
    if "keywords" in id2authorTopics[authorId]:
        keywords = str(id2authorTopics[authorId]["keywords"])
    topics = ""
    if "topics" in id2authorTopics[authorId]:
        topics = str(id2authorTopics[authorId]["topics"])
    prompt_str = prompt_str + "Assuming you are an expert researcher in these areas:\n"
    prompt_str = prompt_str + "Keywords:" + keywords + "\nTopics: "+topics+"\n"
    question = papers_sampled[paperId]["abstract"]

    question_example = []
    option1_example = []
    option2_example = []
    count = 0
    for index,item in enumerate(author_dict):
        question_example.append("Example #"+str(index)+"\nAbstract: "+item["abstract"]+"\nStyle: "+item["result"]+"\nTitle: "+item["title"])

    prompt_str = prompt_str + "I give you some titles and paper abstracts that you've written. Please imitate your style of writing titles. Each example consists of an abstract, the corresponding title, and a description of the writing style and keywords for that title.\n"
    prompt_str = prompt_str + "\n".join(question_example) + "\n"
    prompt_str = prompt_str + "Now you have written this abstract:\n"
    prompt_str = prompt_str + "Abstract:" + question + "\n"
    prompt_str = prompt_str + "Please write a title following your previous style and habits, and keep it clear and accurate:\n"
    prompt_str = prompt_str + "Title: "

    return prompt_str


def run_openai(prompt,key_bundles,i,indice):
    my_dict = {}
    my_dict["role"] = "user"
    my_dict["content"] = prompt
    l = []
    l.append(my_dict)
    result = ""
    try:
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
        result = response.choices[0].message["content"].replace('\n', ' ')
        # print(indice+"\t"+result+"\n")
        return result
    except openai.error.Timeout:
        i = i+1
        print("Timeout",indice)
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        try:
            response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
            result = response.choices[0].message["content"].replace('\n', ' ')
            # print(indice+"\t"+result+"\n")
            return result
        except openai.error.Timeout:
            i = i+1
            print("Timeout",indice)
            key_bundle = key_bundles[i%3]
            openai.api_key, openai.api_base = key_bundle
            try:
                response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
                result = response.choices[0].message["content"].replace('\n', ' ')
                # print(indice+"\t"+result+"\n")
                return result
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                error_list.append(i)
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            error_list.append(i)
        #print(result)
    except openai.error.InvalidRequestError:
        print("InvalidRequestError",indice)
        error_list.append(i)
        #print(result)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        error_list.append(i)
    return result

with open("naive_method_v3.0/single_author_based_title_5_examples.tsv", "wb", buffering=0) as out_file:
    for indice,paperId in enumerate(id2ICLexamples):
        for authorId in id2ICLexamples[paperId]:
            prompt = construct_prompt_cot_agg(id2ICLexamples[paperId][authorId],authorId,paperId)
            # print(prompt)
            # break
            result = run_openai(prompt,key_bundles,i,indice)
            print(indice, result)
            write_str = bytes(str(paperId)+"\t"+str(authorId)+"\t"+result+"\n", 'utf-8')
            out_file.write(write_str)
        

In [12]:
# Zero shot results 
error_list=[]
i = 0
openai.api_key = ""
openai.api_base = "https://gersteinbiocodeeval-eastus2.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future
key_bundles = [
    ('', "https://biocodeeval-openai.openai.azure.com/"),
    ('', "https://gersteinbiocodeeval-eastus2.openai.azure.com/"),
    ('', "https://biocodeeval-openai.openai.azure.com/")
]

    # prompt_construction v1.6
def construct_prompt_cot_agg(author_dict,paperId):
    prompt_str = ""
    question = papers_sampled[paperId]["abstract"]

    prompt_str = prompt_str + "Now you have written this abstract:\n"
    prompt_str = prompt_str + "Abstract:" + question + "\n"
    prompt_str = prompt_str + "Please write a title for this abstract:\n"
    prompt_str = prompt_str + "Title: "

    return prompt_str


def run_openai(prompt,key_bundles,i,indice):
    my_dict = {}
    my_dict["role"] = "user"
    my_dict["content"] = prompt
    l = []
    l.append(my_dict)
    result = ""
    try:
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
        result = response.choices[0].message["content"].replace('\n', ' ')
        # print(indice+"\t"+result+"\n")
        return result
    except openai.error.Timeout:
        i = i+1
        print("Timeout",indice)
        key_bundle = key_bundles[i%3]
        openai.api_key, openai.api_base = key_bundle
        try:
            response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
            result = response.choices[0].message["content"].replace('\n', ' ')
            # print(indice+"\t"+result+"\n")
            return result
        except openai.error.Timeout:
            i = i+1
            print("Timeout",indice)
            key_bundle = key_bundles[i%3]
            openai.api_key, openai.api_base = key_bundle
            try:
                response = openai.ChatCompletion.create(engine="gpt-35-turbo", messages=l, temperature=0.0,request_timeout=30)
                result = response.choices[0].message["content"].replace('\n', ' ')
                # print(indice+"\t"+result+"\n")
                return result
            except Exception as e:
                print(f"An error occurred: {str(e)}")
                error_list.append(i)
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            error_list.append(i)
        #print(result)
    except openai.error.InvalidRequestError:
        print("InvalidRequestError",indice)
        error_list.append(i)
        #print(result)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        error_list.append(i)
    return result

id2result = {}
with open("naive_method_v1.0/zero_shot_results.tsv", "wb", buffering=0) as out_file:
    for indice,paperId in enumerate(id2ICLexamples):
        prompt = construct_prompt_cot_agg(id2ICLexamples[paperId],paperId)
        print(prompt)
        break
        result = run_openai(prompt,key_bundles,i,indice)
        print(indice, result)
        id2result[paperId] = result
        write_str = bytes(str(paperId)+"\t"+result+"\n", 'utf-8')
        out_file.write(write_str)
        

Now you have written this abstract:
Abstract:This book discusses how model-based approaches can improve the daily practice of software professionals. This is known as Model-Driven Software Engineering (MDSE) or, simply, Model-Driven Engineering (MDE). MDSE practices have proved to increase efficiency and effectiveness in software development, as demonstrated by various quantitative and qualitative studies. MDSE adoption in the software industry is foreseen to grow exponentially in the near future, e.g., due to the convergence of software development and business analysis. The aim of this book is to provide you with an agile and flexible tool to introduce you to the MDSE world, thus allowing you to quickly understand its basic principles and techniques and to choose the right set of MDSE instruments for your needs so that you can start to benefit from MDSE right away. The book is organized into two main parts. The first part discusses the foundations of MDSE in terms of basic concepts (

In [ ]:
from rouge import Rouge
import evaluate
pred = []
gold = []
for i in id2result:
    pred.append(id2result[i])
    gold.append(papers_sampled[i]["title"])
def postprocess_text_generation(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def create_metric_rouge():
    rouge_metric = evaluate.load('rouge')
    def compute_metrics(decoded_preds, decoded_labels):
        decoded_preds, decoded_labels = postprocess_text_generation(decoded_preds, decoded_labels)
        result_rouge = rouge_metric.compute(predictions=decoded_preds, references=decoded_labels)
        result = {"rouge-1" : result_rouge["rouge1"], "rouge-L" : result_rouge["rougeL"]}
        return result
    return compute_metrics
metric = create_metric_rouge()
print(metric(pred, gold))